<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>

## 1. Introduction

In this notebook, we will create a preprocessing and feature engineering pipeline with [Rapids cuDF](https://github.com/rapidsai/cudf) and [Merlin NVTabular](https://github.com/NVIDIA/NVTabular) libraries to prepare our dataset for session-based recommendation model training. 

NVTabular is a feature engineering and preprocessing library for tabular data that is designed to easily manipulate terabyte scale datasets and train deep learning (DL) based recommender systems.

Our main goal is to create sequential features. In order to do that, we are going to perform the following:

- Categorify categorical features with `Categorify()` op
- Create temporal features with a `user-defined custom` op and `Lambda` op
- Transform continuous features using `Log` and `Normalize` ops
- Group all these features together at the session level sorting the interactions by time with `Groupby`
- Finally export the preprocessed datasets to parquet files by hive-partitioning.

### 1.1. Dataset

In our hands-on exercise notebooks we are going to use a subset of the publicly available [eCommerce dataset](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store). The ecommeerce behavior data contains 7 months of data (from October 2019 to April 2020) from a large multi-category online store. Each row in the file represents an event. All events are related to products and users. Each event is like many-to-many relation between products and users.

Data collected by Open CDP project and the source of the dataset is [REES46 Marketing Platform](https://rees46.com/).

## 2. Import Libraries

In [1]:
import os

import numpy as np 
import cupy as cp
import glob

import cudf
import nvtabular as nvt
from nvtabular import ColumnSelector

## 3. Set up Input and Output Data Paths

In [2]:
# define data path about where to get our data
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/dli/task/data/")

## 4. Read the Input Parquet file

We already performed certain preprocessing steps on the first month (Oct-2019) of the raw dataset. You can visit this notebook [01-preprocess.ipynb](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/tutorial/01-preprocess.ipynb) to go through the code for the following preprocessing steps : <br>

- we created `event_time_ts` column from `event_time` column which shows the time when event happened at (in UTC).
- we created `prod_first_event_time_ts` column which indicates the timestamp that an item was seen first time.
- we removed the rows where the `user_session` is Null. As a result, 2 rows were removed.
- we categorified the `user_session` column, so that it now has only integer values.
- we removed consequetively repeated (user, item) interactions. For example, an original session with `[1, 2, 4, 1, 2, 2, 3, 3, 3]` product interactions has become `[1, 2, 4, 1, 2, 3]` after removing the repeated interactions on the same item within the same session.

Even though the original dataset contains 7 months data files, we are going to use the first seven days of the `Oct-2019.csv` ecommerce dataset. We use cuDF to read the parquet file. 

In [3]:
%%time
df = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, 'Oct-2019.parquet'))  
df.head(5)

CPU times: user 721 ms, sys: 444 ms, total: 1.17 s
Wall time: 1.16 s


user_session event_type  product_id          category_id  \
0            46       view     3701349  2053013565983425517   
1            50       view    26403432  2053013563651392361   
2            50       view    26400653  2053013563651392361   
3            50       view    26401406  2053013563651392361   
4            50       view    26400470  2053013563651392361   

                   category_code     brand   price    user_id  event_time_ts  \
0  appliances.environment.vacuum  scarlett  102.71  526012229     1569984606   
1                           <NA>      <NA>  109.66  555257976     1569959048   
2                           <NA>   lucente  227.81  555257976     1569959124   
3                           <NA>   lucente  111.46  555257976     1569959163   
4                           <NA>      <NA>  173.75  555257976     1569959195   

   prod_first_event_time_ts  
0                1569898527  
1                1569898887  
2                1569900969  
3                1569910560  
4                1569919542

In [4]:
df.shape

(6390928, 10)

Let's check if there is any column with nulls.

In [5]:
df.isnull().any()

user_session                False
event_type                  False
product_id                  False
category_id                 False
category_code                True
brand                        True
price                       False
user_id                     False
event_time_ts               False
prod_first_event_time_ts    False
dtype: bool

We see that `'category_code'` and `'brand'` columns have null values, and in the following cell we are going to fill these nulls with via categorify op, and then all categorical columns will be encoded to continuous integers. Note that we add `start_index=1` in the `Categorify op` for the categorical columns, the reason for that we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

## 5. Initialize NVTabular Workflow

### 5.1. Categorical Features Encoding

In [6]:
# categorify features 
cat_feats = ['user_session', 'category_code', 'brand', 'user_id', 'product_id', 'category_id', 'event_type'] >> nvt.ops.Categorify(start_index=1)

### 5.2. Extract Temporal Features

In [7]:
# create time features
session_ts = ['event_time_ts']

session_time = (
    session_ts >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

sessiontime_weekday = (
    session_time >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(name ='et_dayofweek')
)

Now let's create cycling features from the `sessionTime_weekday` column. We would like to use the temporal features (hour, day of week, month, etc.) that have inherently cylical characteristic. We represent the day of week as a cycling features (sine and cosine), so that it can be represented in a continuous spae. That way, the difference between the representation of two different days is the same, in other words, with cyclical features we can convey closeness between data. You can read more about it [here](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/).

In [8]:
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

def get_cycled_feature_value_cos(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_cos

In [9]:
weekday_sin = sessiontime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_sin')
weekday_cos= sessiontime_weekday >> (lambda col: get_cycled_feature_value_cos(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_cos')

### 5.2.1 Add Product Recency feature

- Let's define a custom op to calculate product recency in days

In [10]:
from nvtabular.ops import Operator

class ItemRecency(Operator):
    def transform(self, columns, gdf):
        for column in columns.names:
            col = gdf[column]
            item_first_timestamp = gdf['prod_first_event_time_ts']
            delta_days = (col - item_first_timestamp) / (60*60*24)
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf
            
    def output_column_names(self, columns):
        return ColumnSelector([column + "_age_days" for column in columns.names])

    def dependencies(self):
        return ["prod_first_event_time_ts"]
    
    
recency_features = ['event_time_ts'] >> ItemRecency() 
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(name='product_recency_days_log_norm')

In [11]:
time_features = (
    session_time +
    sessiontime_weekday +
    weekday_sin +
    weekday_cos +
    recency_features_norm
)

### 5.3. Normalize Continuous Features¶

In [12]:
# Smoothing price long-tailed distribution and applying standardization
price_log = ['price'] >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(name='price_log_norm')

In [13]:
# Relative price to the average price for the category_id
def relative_price_to_avg_categ(col, gdf):
    epsilon = 1e-5
    col = ((gdf['price'] - col) / (col + epsilon)) * (col > 0).astype(int)
    return col
    
avg_category_id_pr = ['category_id'] >> nvt.ops.JoinGroupby(cont_cols =['price'], stats=["mean"]) >> nvt.ops.Rename(name='avg_category_id_price')
relative_price_to_avg_category = avg_category_id_pr >> nvt.ops.LambdaOp(relative_price_to_avg_categ, dependency=['price']) >> nvt.ops.Rename(name="relative_price_to_avg_categ_id")

### 5.4. Grouping interactions into sessions

#### Aggregate by session id and creates the sequential features

In [14]:
groupby_feats = ['event_time_ts', 'user_session'] + cat_feats + time_features + price_log + relative_price_to_avg_category

In [15]:
# Define Groupby Workflow
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["user_session"], 
    sort_cols=["event_time_ts"],
    aggs={
        'user_id': ['first'],
        'product_id': ["list", "count"],
        'category_code': ["list"],  
        'event_type': ["list"], 
        'brand': ["list"], 
        'category_id': ["list"], 
        'event_time_ts': ["first"],
        'event_time_dt': ["first"],
        'et_dayofweek_sin': ["list"],
        'et_dayofweek_cos': ["list"],
        'price_log_norm': ["list"],
        'relative_price_to_avg_categ_id': ["list"],
        'product_recency_days_log_norm': ["list"]
        },
    name_sep="-")

- Select columns which are list

In [16]:
groupby_features_list = groupby_features['product_id-list',
        'category_code-list',  
        'event_type-list', 
        'brand-list', 
        'category_id-list', 
        'et_dayofweek_sin-list',
        'et_dayofweek_cos-list',
        'price_log_norm-list',
        'relative_price_to_avg_categ_id-list',
        'product_recency_days_log_norm-list']

In [17]:
SESSIONS_MAX_LENGTH = 20 
MINIMUM_SESSION_LENGTH = 2

We truncate the sequence features in length according to sessions_max_length param, which is set as 20 in our example.

In [18]:
groupby_features_trim = groupby_features_list >> nvt.ops.ListSlice(0,SESSIONS_MAX_LENGTH) >> nvt.ops.Rename(postfix = '_seq')

- Create a `day_index` column in order to partition sessions by day when saving the parquet files.

In [19]:
# calculate session day index based on 'timestamp-first' column
day_index = ((groupby_features['event_time_dt-first'])  >> 
    nvt.ops.LambdaOp(lambda col: (col - col.min()).dt.days +1) >> 
    nvt.ops.Rename(f = lambda col: "day_index")
)

- Select certain columns to be used in model training

In [20]:
selected_features = groupby_features['user_session', 'product_id-count'] + groupby_features_trim + day_index

- Filter out the session that have less than 2 interactions.

In [21]:
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["product_id-count"] >= MINIMUM_SESSION_LENGTH)

- Initialize the NVTabular dataset object and workflow graph.

NVTabular's preprocessing and feature engineering workflows are directed graphs of operators. When we initialize a Workflow with our pipeline, workflow organizes the input and output columns.

In [22]:
# avoid numba warnings
from numba import config
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [23]:
dataset = nvt.Dataset(df)

workflow = nvt.Workflow(filtered_sessions)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf()

Above, we created an NVTabular Dataset object using our input dataset. Then, we calculate statistics for this workflow on the input dataset, i.e. on our training set, using the workflow.fit() method so that our Workflow can use these stats to transform any given input.

Let's print the head of our preprocessed dataset. You can notice that now each example (row) is a session and the sequential features with respect to user interactions were converted to lists with matching length.

In [24]:
sessions_gdf.head(3)

user_session  product_id-count  \
0             2               779   
1             3               316   
2             4               277   

                                 product_id-list_seq  \
0  [19064, 52057, 13290, 11446, 15835, 879, 633, ...   
1  [252, 2801, 5399, 1074, 252, 355, 327, 319, 34...   
2  [765, 353, 1360, 1965, 2204, 3129, 726, 861, 9...   

                              category_code-list_seq  \
0  [1, 1, 1, 1, 1, 12, 12, 12, 12, 12, 12, 12, 12...   
1  [1, 17, 17, 15, 1, 1, 17, 31, 17, 17, 17, 17, ...   
2  [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...   

                                 event_type-list_seq  \
0  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
1  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
2  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   

                                      brand-list_seq  \
0  [171, 120, 231, 392, 562, 20, 9, 20, 295, 143,...   
1  [1, 1, 1, 1, 1, 50, 1, 1, 36, 1, 1, 36, 50, 1,...   
2  [448, 114, 1, 20, 20, 72, 114, 143, 20, 141, 7...   

                                category_id-list_seq  \
0  [3, 3, 3, 3, 3, 17, 17, 17, 17, 17, 17, 17, 17...   
1  [234, 36, 36, 30, 234, 52, 36, 48, 36, 36, 36,...   
2  [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...   

                           et_dayofweek_sin-list_seq  \
0  [0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...   
1  [0.43388295, 0.43388295, 0.43388295, 0.4338829...   
2  [0.43388295, 0.43388295, 0.43388295, 0.4338829...   

                           et_dayofweek_cos-list_seq  \
0  [-0.22252177, -0.22252177, -0.22252177, -0.222...   
1  [-0.90096927, -0.90096927, -0.90096927, -0.900...   
2  [-0.90096927, -0.90096927, -0.90096927, -0.900...   

                             price_log_norm-list_seq  \
0  [-0.60630524, -0.59222364, -0.5865736, -0.9532...   
1  [0.7637999, 0.40693888, 0.25858778, 0.01305765...   
2  [-1.7807875, -0.5645757, -0.04535111, -0.43499...   

             relative_price_to_avg_categ_id-list_seq  \
0  [0.035192717967850506, 0.053910707681354364, 0...   
1  [0.0006990395296892537, -0.048753441585919946,...   
2  [-0.8327209899254826, -0.191232403680255, 0.56...   

              product_recency_days_log_norm-list_seq  day_index  
0  [-2.2660863, -2.2660863, -2.2657661, -2.266086...          1  
1  [-0.85815096, -0.9379311, -1.0066847, -0.93632...          2  
2  [-0.79925084, -0.78641766, -0.82284164, -0.795...          2

In [25]:
workflow.output_schema.column_names

['user_session',
 'product_id-count',
 'product_id-list_seq',
 'category_code-list_seq',
 'event_type-list_seq',
 'brand-list_seq',
 'category_id-list_seq',
 'et_dayofweek_sin-list_seq',
 'et_dayofweek_cos-list_seq',
 'price_log_norm-list_seq',
 'relative_price_to_avg_categ_id-list_seq',
 'product_recency_days_log_norm-list_seq',
 'day_index']

- Save NVTabular workflow to load at the inference step.

In [26]:
workflow_path = os.path.join(INPUT_DATA_DIR, 'workflow_etl')
workflow.save(workflow_path)

## 6. Exporting data

We export dataset to parquet partioned by the session `day_index` column.

In [27]:
# define partition column
PARTITION_COL = 'day_index'

# define output_folder to store the partitioned parquet files
OUTPUT_FOLDER = os.environ.get("OUTPUT_FOLDER", INPUT_DATA_DIR + "sessions_by_day")
!mkdir -p $OUTPUT_FOLDER

In this section we are going to create a folder structure as shown below. As we explained above, this is just to structure parquet files so that it would be easier to do incremental training and evaluation.

```
/sessions_by_day/
|-- 1
|   |-- train.parquet
|   |-- valid.parquet
|   |-- test.parquet

|-- 2
|   |-- train.parquet
|   |-- valid.parquet
|   |-- test.parquet
```

`gpu_preprocessing` function converts the process df to a Dataset object and write out hive-partitioned data to disk.

In [28]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_FOLDER,
                       partition_col=PARTITION_COL,
                       timestamp_col='user_session', 
                      )

Creating time-based splits: 100%|██████████| 7/7 [00:02<00:00,  3.08it/s]


In [29]:
# check out the OUTPUT_FOLDER
!ls $OUTPUT_FOLDER

1  2  3  4  5  6  7


## 7. Wrap Up 

That's it! We finished our first task. We reprocessed our dataset and created new features to train a session-based recommendation model. Please run the cell below to shut down the kernel before moving on to the next notebook.

In [30]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}